In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPool1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

In [5]:

!wget https://lazyprogrammer.me/course_files/spam.csv
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df.columns = ['labels', 'data']
df.head()

--2024-06-14 04:49:09--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv.3’

spam.csv.3          100%[===================>] 491.86K  --.-KB/s    in 0.05s   

2024-06-14 04:49:09 (10.5 MB/s) - ‘spam.csv.3’ saved [503663/503663]



,labels,data
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df['b_labels'] = df['labels'].map({'ham': 0, 'spam': 1})
Y = df['b_labels'].values
Y

array([0, 0, 1, ..., 0, 0, 0])

In [7]:
df_train, df_test, Ytrain, Ytest = train_test_split(df['data'], Y, test_size=0.33)

In [8]:
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

In [9]:
word2idx = tokenizer.word_index
V = len(word2idx)
V

7236

In [10]:
data_train = pad_sequences(sequences_train)
data_train.shape
T = data_train.shape[1]

In [11]:
data_test = pad_sequences(sequences_test, maxlen=T)
data_test.shape

(1839, 189)

In [12]:
D = 20

i = Input(shape=(T,))
x = Embedding(V + 1, D)(i)
x = Conv1D(32, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPool1D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)

In [14]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
r = model.fit(data_train, Ytrain, epochs=5, validation_data=(data_test, Ytest))

Epoch 1/5
117/117 [==============================] - 6s 31ms/step - loss: 0.3746 - accuracy: 0.8690 - val_loss: 0.2514 - val_accuracy: 0.8597
Epoch 2/5
117/117 [==============================] - 2s 18ms/step - loss: 0.0751 - accuracy: 0.9759 - val_loss: 0.0943 - val_accuracy: 0.9728
Epoch 3/5
117/117 [==============================] - 2s 18ms/step - loss: 0.0142 - accuracy: 0.9968 - val_loss: 0.0792 - val_accuracy: 0.9804
Epoch 4/5
117/117 [==============================] - 2s 18ms/step - loss: 0.0041 - accuracy: 0.9989 - val_loss: 0.0878 - val_accuracy: 0.9804
Epoch 5/5
117/117 [==============================] - 2s 18ms/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.0843 - val_accuracy: 0.9815
